## Lexical Features (table-retrieval LTR baseline)

In [1]:
import nltk
import numpy as np
import pandas as pd

In [58]:
tables = pd.read_json(r'data/tables/re_tables-0875.json')
features = pd.read_csv(r'data/features/features.txt')
qrels = pd.read_csv(r'data/queries/qrels.txt', sep='\t', header=None)
queries = pd.read_csv(r'data/queries/queries.txt', header=None)
queries = pd.DataFrame([row[0][row[0].find(' ') + 1:] for index, row in queries.iterrows()])

In [11]:
# list of stop words from nltk english corpus
sw = nltk.corpus.stopwords.words('english')

In [12]:
features.head()

,query_id,query,table_id,row,col,nul,in_link,out_link,pgcount,tImp,...,cmax,csum,cavg,csim,remax,resum,reavg,resim,query_l,rel
0,1,world interest rates Table,table-0875-680,8,2,0,31,21,51438,1.0,...,0.666667,5.291894,0.048108,0.354686,0.241209,3.716354,0.033785,0.281130,4,0
1,1,world interest rates Table,table-1020-619,4,3,0,18,0,324,1.0,...,1.000000,11.116121,0.101056,0.718895,1.000000,8.075247,0.073411,0.710250,4,0
2,1,world interest rates Table,table-0288-531,3,5,0,23,22,26419,0.5,...,0.000000,0.000000,0.000000,0.000000,0.067373,0.365818,0.003326,0.033680,4,0
3,1,world interest rates Table,table-0288-530,4,5,1,23,22,26419,0.5,...,0.000000,0.000000,0.000000,0.000000,0.067373,0.365818,0.003326,0.033680,4,0
4,1,world interest rates Table,table-1000-57,2,2,0,38,1,2268,1.0,...,1.000000,10.147388,0.092249,0.372667,0.226134,4.564622,0.041497,0.279899,4,0


In [13]:
print(features.iloc[1])

query_id                                  1
query            world interest rates Table
table_id                     table-1020-619
row                                       4
col                                       3
nul                                       0
in_link                                  18
out_link                                  0
pgcount                                 324
tImp                                      1
tPF                             0.000786473
leftColhits                               0
SecColhits                                0
bodyhits                                  0
PMI                               -0.231049
qInPgTitle                                0
qInTableTitle                      0.285714
yRank                                   100
csr_score                       8.66373e-10
idf1                                29.6279
idf2                                24.1356
idf3                                27.1006
idf4                            

### Preprocessing

In [59]:
# save table ids for later use
table_ids = features['table_id']

In [60]:
# one-hot encode the data using pandas get_dummies
features = pd.get_dummies(features, columns = ['table_id'])

In [61]:
features.drop(['query', 'max', 'sum', 'avg', 'sim', 'emax', 'esum', 'eavg', 'esim', 'cmax', 'csum', 'cavg', 'csim', 'remax', 'resum', 'reavg', 'resim'], axis = 1, inplace = True)

In [62]:
# extract labels from features (labels are the values we want to predict)
labels = np.array(features['rel'])

# remove labels from features
features.drop(['rel'], axis = 1, inplace = True)

# save feature names for later use
feature_columns = list(features.columns)

# convert features to numpy array
features = np.array(features)

### Training and Testing Sets

In [63]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.20, random_state = 42)

In [64]:
print(f'Training Features Shape: {train_features.shape}')
print(f'Training Labels Shape: {train_labels.shape}')
print(f'Testing Features Shape: {test_features.shape}')
print(f'Testing Labels Shape: {test_labels.shape}')

Training Features Shape: (2496, 2956)
Training Labels Shape: (2496,)
Testing Features Shape: (624, 2956)
Testing Labels Shape: (624,)


### Model Training (random forests)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, max_depth=3, n_jobs = 10, random_state = 42)

# train the model on training data
rf.fit(train_features, train_labels)

#### Save model to file

In [ ]:
from joblib import dump
# random_forest_1000_none.joblib: n_estimators = 1000, max_depth = None
# random_forest_1000_3.joblib: n_estimators = 1000, max_depth = 3
dump(rf, 'random_forest_1000_3.joblib')

#### Load model from file

In [ ]:
from joblib import load
rf = load('random_forest.joblib')

#### Grid search

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# create the parameter grid
param_grid = {
    'bootstrap': [True],
    'max_depth': [3, 5, None],
    'n_estimators': [100, 500, 1000, 1500, 2000]
}

# create a based model
rf = RandomForestRegressor()

# instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 2)

In [ ]:
# fit the grid search to the data
grid_search.fit(train_features, train_labels)

### Results

In [ ]:
from sklearn import metrics

predictions = rf.predict(test_features)
print(f'mean square error  : {metrics.mean_squared_error(test_labels, predictions)}')
print(f'mean absolute error: {metrics.mean_absolute_error(test_labels, predictions)}')

In [ ]:
predictions = rf.predict(features)
print(f'mean square error  : {metrics.mean_squared_error(labels, predictions)}')
print(f'mean absolute error: {metrics.mean_absolute_error(labels, predictions)}')

In [ ]:
# generate results in trec_eval format
raw = pd.read_csv(r'data/features/features.txt')
res = {
    'query-id': list(),
    'q0': list(),
    'document-id': list(),
    'rank': list(),
    'score': list(),
    'name': list()
}

for index, row in raw.iterrows():
    res['query-id'].append(row['query_id'])
    res['q0'].append('Q0')
    res['document-id'].append(row['table_id'])
    res['rank'].append(0)
    res['score'].append(predictions[index])
    res['name'].append('STANDARD')

In [ ]:
df_res = pd.DataFrame.from_dict(res)

In [ ]:
df_res.head()

In [ ]:
# save results to .txt file (for running trec_eval comparison)
df_res.to_csv('results_1000_3.txt', sep=' ', index=False, header=False)

### Query features

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

# QLEN
def get_qlen(query):
    return len(query.split(' '))

# IDF
def get_idf(query, field):
    # instantiate count vectorizer
    cv=CountVectorizer(field, stop_words=sw)
    # this steps generates word counts for the words in your docs
    word_count_vector=cv.fit_transform(field)
    # instantiate tfidf transformer (with use_idf true in order to compute idf scores)
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    # compute the idf scores
    tfidf_transformer.fit(word_count_vector)
    # compute the sum of idf scores for all query terms
    score = sum([tfidf_transformer.idf_[cv.get_feature_names().index(term)] for term in query.split(' ')])
    # return idf score
    return score

### Lexical features

In [ ]:
import nltk
import numpy as np
import pandas as pd
import logging
import wikipediaapi
import pageviewapi.period
from wikitables import import_tables
logging.getLogger('wikitables').setLevel(logging.ERROR)

wiki = wikipediaapi.Wikipedia('en')

for i in range(1, 1001):
    print(f'processing file {i}')
    prefix = '0' * (4 - len(str(i)))
    file = prefix + str(i)
    out_features = 'data/features/table-features-' + file + '.txt'
    table = 'data/tables/re_tables-' + file +'.json'
    tmp = pd.read_json(table)
    tmpT = tmp.T
    
    fields = {
        'page_titles': set(),
        'section_titles': set(),
        'table_captions': set(),
        'table_bodies': set()
    }
    
    features = {
        'table_id': list(),
        'rows': list(),
        'cols': list(),
        'nulls': list(),
        'inlinks': list(),
        'outlinks': list(),
        'views': list(),
        'table_imp': list(),
        'table_fraction': list()
    }
    
    i = 0
    for index, row in tmpT.iterrows():
        table_id = tmp.iloc[0].index[i]
        rows = row['numDataRows']
        cols = row['numCols']
        title = row['pgTitle']
        caption = row['caption']
        data = row['data']
        section_title = [item.lower() for item in row['title']]
        fields['section_titles'].update(section_title)
        
        inlinks = 0; outlinks = 0; views = 0; table_imp = 0; text_len = 0; chars = 0; nulls = 0; page_tables = 0
        
        for entry in data:
            for item in entry:
                fields['table_bodies'].update({item.lower()})
                if len(item) == 0:
                    nulls += 1
                chars += len(item)
                
        page = wiki.page(title)
        if page.exists():
            inlinks = len(page.backlinks)
            outlinks = len(page.links)
            try:
                views = pageviewapi.period.sum_last('en.wikipedia', title, last=365, access='all-access', agent='all-agents')
            except:
                pass
            try:
                page_tables = len(import_tables(title))
            except:
                pass
            table_imp = 1 / (page_tables + 1)
            text_len = len(page.text)
        
        table_fraction = chars / (text_len + 1)
        
        fields['page_titles'].update({title.lower()})
        fields['table_captions'].update({caption.lower()})
        
        features['table_id'].append(table_id)
        features['rows'].append(rows)
        features['cols'].append(cols)
        features['nulls'].append(nulls)
        features['inlinks'].append(inlinks)
        features['outlinks'].append(outlinks)
        features['views'].append(views)
        features['table_imp'].append(table_imp)
        features['table_fraction'].append(table_fraction)
        i += 1
        if i % 100 == 0:
            print(f'done processing {i} tables!')
    df_features = pd.DataFrame.from_dict(features)
    df_features.to_csv(out_features, sep=' ', index=False, header=True)
    
    out_fields = 'data/fields/page_titles/table-fields-' + file + '.txt'
    df_fields = pd.DataFrame.from_dict(fields['page_titles'])
    df_fields.to_csv(out_fields, sep=' ', index=False, header=True)
    
    out_fields = 'data/fields/section_titles/table-fields-' + file + '.txt'
    df_fields = pd.DataFrame.from_dict(fields['section_titles'])
    df_fields.to_csv(out_fields, sep=' ', index=False, header=True)
    
    out_fields = 'data/fields/table_captions/table-fields-' + file + '.txt'
    df_fields = pd.DataFrame.from_dict(fields['table_captions'])
    df_fields.to_csv(out_fields, sep=' ', index=False, header=True)
    
    out_fields = 'data/fields/table_bodies/table-fields-' + file + '.txt'
    df_fields = pd.DataFrame.from_dict(fields['table_bodies'])
    df_fields.to_csv(out_fields, sep=' ', index=False, header=True)
    
    print(f'done processing file {i}')
print(f'done processing all tables!')

In [ ]:
df_features = pd.DataFrame.from_dict(features)
df_features.to_csv('table-features-1.txt', sep=' ', index=False, header=True)

In [ ]:
print(features)
print(page_titles)
print(section_titles)
print(table_captions)
print(table_bodies)

In [ ]:
import bigjson.bigjson as bj

with open('tables.json', 'rb') as f:
    reader = bj.FileReader(f, 'utf-8')
    i = reader.read(True, False)
    
print(i.items())

In [ ]:
with open('tableMentions.json', 'rb') as f:
    reader = bj.FileReader(f, 'utf-8')
    j = reader.read(True, True)
    
print(j.items())

In [ ]:
# stack the data frames on top of each other
# vertical_stack = pd.concat([df_1, df_2], axis=0)
# newtmp = tmp.set_index('table_id').T.to_dict('list')
# tmp1 = pd.DataFrame.from_dict(newtmp).T